<a href="https://colab.research.google.com/github/kunai-3txk/colab/blob/main/20220522_kame_33_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

XGBoostの概要とPython実装をわかりやすく解説【機械学習入門32】

https://datawokagaku.com/xgboost/

---



過学習を抑えるための工夫
XGBoostは，先ほどの正則化項以外にも色々と過学習を抑えるための工夫をしています

Shrinkage(縮小)

それぞれの決定木の結果に係数η(０〜1)をつけることで，それぞれの決定木の影響を小さく(縮小=shrinkage)します．

Column Subsampling

一つ一つの決定木の分割に使う特徴量をランダムに選択します．(つまり，一部の特徴量のみ分割して決定木を作ります)

---



PythonでXGBoostを使う

回帰モデルには xgboost.XGBRegressor を，分類モデルには xgboost.XGBClassifier を使います．どちらも仕様はほとんど同じです

---



In [ ]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
# df = df.dropna()
X = df.loc[:, (df.columns!='survived') & (df.columns!='alive')]
X = pd.get_dummies(X, drop_first=True)
y = df['survived']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

今まで通りなので説明は省きますが，実はXGBoostは欠損値を対処するアルゴリズムが組み込まれているので，欠損値をdropしたり代入する必要がなく，欠損値があるデータをそのままモデルに学習させることができます．

---



XGBClassifier() のインスタンス生成時のパラメータは以下があります．

n_estimators : ブースティングの回数(=学習する決定木の数)．デフォルトは100．

learning_rate : shrinkageのη．デフォルトは0.3なので少し高め．(低スペックマシンでの実行を想定しているのだと思われます．)

max_depth : 決定木の最高の深さ．デフォルトは6

eval_metric : ブースティング時の各イテレーション時に使う評価指標(特に後述するearly stoppingに使用). sklearn.metricsのメソッドを渡すか，自作してもOK．デフォルトは 'logloss'

early_stopping_rounds : early stoppingする際の，最低限ブースティングのイテレーションをする回数

---
early stoppingというのは，ブースティングのイテレーション時に評価指標がこれ以上下がらなくなったら自動で学習をやめてくれます．この仕組みにより， n_estimators や learning_rate のパラメータを探索する必要がほとんどなくなります． n_estimators には高い数字を， learning_rate には低い数字をセットしておけば，あとはearly stoppingで学習を回せば最も精度が高くなったところで自動で学習をやめてくれます．

---




In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=1000)

.fit()のよく使う引数は以下です．

eval_set : (X, y)のリストで，これを渡すと，ブースティング時の各イテレーションごとにこのデータセットを使って評価してくれる．

verbose : Trueにすると，各イテレーションでの評価指標などを表示し，イテレーションの軌跡を確認できる．

---



In [ ]:
#今回は，テストデータを各イテレーションで評価するデータとして使います．
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=True,early_stopping_rounds=10)
#model.fit(X_train, y_train,verbose=True)


[0]	validation_0-error:0.253731
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.253731
[2]	validation_0-error:0.201493
[3]	validation_0-error:0.197761
[4]	validation_0-error:0.197761
[5]	validation_0-error:0.197761
[6]	validation_0-error:0.186567
[7]	validation_0-error:0.186567
[8]	validation_0-error:0.179104
[9]	validation_0-error:0.179104
[10]	validation_0-error:0.179104
[11]	validation_0-error:0.190299
[12]	validation_0-error:0.190299
[13]	validation_0-error:0.190299
[14]	validation_0-error:0.190299
[15]	validation_0-error:0.190299
[16]	validation_0-error:0.19403
[17]	validation_0-error:0.19403
[18]	validation_0-error:0.190299
Stopping. Best iteration:
[8]	validation_0-error:0.179104



XGBClassifier(early_stopping_rounds=10, n_estimators=1000)

In [ ]:
from sklearn import metrics
y_pred = model.predict_proba(X_test)
metrics.log_loss(y_test, y_pred)

0.49012428766755917

特徴量の重要度
ランダムフォレスト同様，XGBoostも特徴量の重要度を出すことができます．( xgboost.plot_importance(model) でも同様なplotをしてくれる便利な関数が実装されています．)


---

